In [199]:
import pandas as pd
import re
import requests

data_raw = pd.read_csv("data_raw/participantes.csv", sep = ";").drop(axis=1, labels=["Unnamed: 0"])

Unindo colunas de nascimento e renomeando

In [63]:
cols = ['Data de nascimento', 'Idade', 'Data de Nascimento']
data = data_raw.assign(
    data_nascimento = lambda x: x['Data de nascimento'].fillna("") +
                                x['Data de Nascimento'].fillna("") +
                                x['Idade'].fillna("")
).drop(cols, axis = 1).drop("Ref.", axis = 1)
data = data.rename(columns = {"Participante": 'nome', "Ocupação": "ocupacao", "Resultado":"resultado"})

Corrigindo localidade de origem

In [191]:
data_local = data.assign(
    municipio = lambda x: x["Origem"].str.split(",").str[0],
    pais = lambda x: x["Origem"].str.split(",").str[1]
).drop(["Origem"], axis = 1)

data_local['pais'] = data_local['pais'].fillna("Exterior")

Corrigindo Resultado

In [140]:
num_participantes = data_local.groupby(["edicao"]).agg(
    n_participantes = ("edicao", "count")
)

participantes_edicao = data_local.join(num_participantes, on ='edicao').assign(
    resultado = lambda x: x['resultado'].str.split("em").str[0]
)

In [188]:
def arrumar_classificacao(row):
    # texto, n_participantes

    if any(i in row['resultado'] for i in ["Vencedor", "Vencedora"]):
        row['resultado'] = re.sub("Ven.*", "1", row['resultado'])
    elif any(i in row['resultado'] for i in ["eliminado", "eliminada"]):
        lugar = re.sub("ª eli.*|º eli.*", "", row['resultado'])
        lugar = row['n_participantes'] + 1 - int(lugar)
        row['resultado'] = str(lugar)
    elif "lugar" in row['resultado']: 
        row['resultado'] = row['resultado'][0]
    else:
        row['resultado'] = "99" 
    
    return row['resultado']

participantes_edicao['classificacao'] = participantes_edicao.apply(arrumar_classificacao, axis = 1)
participantes_edicao = participantes_edicao.drop(['resultado'], axis = 1)

- Gênero #Não rodar esse codigo

In [216]:
def genero(row):
    key = "usei o gender-api"
    primeiro_nome = row['nome'].split(" ")[0]
    url = f"https://gender-api.com/get?name={primeiro_nome}&key={key}"
    gender_api = requests.get(url)
    gender_json = eval(gender_api.text)
    gender = gender_json["gender"]

    return gender

participantes_edicao['genero'] = participantes_edicao.apply(genero, axis = 1)

In [225]:
participantes_edicao.to_csv("data_raw/participantes_genero.csv", sep = ";", index = False)

In [226]:
participantes_genero = pd.read_csv("data_raw/participantes_genero.csv", sep = ";")

In [227]:
participantes_genero

,nome,ocupacao,edicao,data_nascimento,municipio,pais,n_participantes,classificacao,genero
0,Kleber de Paula Pedra (Bambam),Dançarino,1,14/02/1978,Campinas,São Paulo,12,1,male
1,Vanessa Melanie Pascale Ekypenyong Walborn,Modelo,1,28/11/1974,Rio de Janeiro,Rio de Janeiro,12,2,female
2,André Batista de Carvalho,Professor de canto,1,28/11/1974,Rio de Janeiro,Rio de Janeiro,12,3,male
3,Antonio Sergio Tavares Campo,Cabeleireiro,1,06/12/1972,Angola,Exterior,12,4,male
4,Alessandra Begliomini,Empresária,1,05/12/1974,São Paulo,São Paulo,12,5,female
...,...,...,...,...,...,...,...,...,...
357,Bárbara Heck,Relações públicas e modelo,22,13/07/1992,Novo Hamburgo,Rio Grande do Sul,22,19,female
358,Vitória Nascimento Câmara (Maria),Atriz e cantora,22,18/07/2000,Rio de Janeiro,Rio de Janeiro,22,99,female
359,Naiara de Fátima Azevedo,Cantora,22,30/10/1989,Campo Mourão,Paraná,22,20,female
360,Rodrigo Abrão de Carvalho Mussi Ivo,Gerente comercial,22,20/09/1985,São José dos Campos,São Paulo,22,21,male
